<a href="https://colab.research.google.com/github/jorcisai/ARF/blob/master/HuggingFace/06-Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning

Motivation:
* Adaptation of pre-trained Transformer models to a specific task
* Leveraging models trained on massive datasets
* Saving time and power

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Taking a look at the content of the Europarl-ST dataset having English as a source language

In [8]:
from datasets import load_dataset

raw_datasets = load_dataset("tj-solergibert/Europarl-ST-processed-mt-en")
print(raw_datasets)

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 602605
    })
    test: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 86170
    })
    valid: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 81968
    })
})


Filtering Europarl-ST only for English into Spanish:

In [12]:
lang="es"
lang_id = raw_datasets["train"].features["dest_lang"].names.index(lang)
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: x["dest_lang"] == lang_id)

A good practice is to take a small random sample to get a quick feel for the type of data you’re working with. In Datasets, we can create a random sample by chaining the Dataset.shuffle() and Dataset.select() functions together:

In [15]:
raw_sample = raw_datasets["train"].shuffle(seed=13).select(range(1000))
# Peek at the first few examples
raw_sample[:3]

{'source_text': ['the general issues of religious freedom, this is an absolute scandal and I welcome the fact that a number of Christian bishops from Iraq are to travel to Strasbourg in December',
  'The Commission fully supports the work of the International Criminal Tribunal for Yugoslavia, the ICTY.',
  'Another boost will come following the wonderful achievement of Swansea City Football Club – and I say this for the benefit of my colleague Richard Howitt – who have just deservedly reached the Premier Football League after a fantastic season.'],
 'dest_text': ['de los problemas de libertad religiosa, esto es un escándalo absoluto y celebro que varios obispos cristianos iraquíes vayan a viajar a Estrasburgo en diciembre',
  'La Comisión apoya sin reservas el trabajo del Tribunal Penal Internacional para Yugoslavia, el TPIY.',
  'Llegará otro impulso a raíz del maravilloso logro del — y lo digo en beneficio de mi colega Richard Howitt —, que acaba de alcanzar merecidamente la tras una

In [14]:
from transformers import AutoTokenizer

checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer(raw_datasets["train"][1]["source_text"],raw_datasets["train"][1]["dest_text"])

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


{'input_ids': [9155, 347, 714, 8192, 261, 2956, 20276, 783, 259, 27155, 484, 798, 12190, 267, 287, 48906, 304, 287, 259, 32443, 304, 287, 35893, 2746, 305, 287, 259, 155924, 2302, 304, 109667, 347, 57675, 106543, 4196, 281, 5411, 288, 4996, 35893, 270, 5454, 261, 259, 31745, 60093, 288, 287, 690, 178535, 272, 304, 259, 9953, 57675, 260, 1, 431, 8810, 980, 393, 28486, 268, 1444, 865, 128163, 21748, 1559, 1298, 35072, 68768, 267, 283, 11126, 931, 426, 259, 4428, 20344, 269, 283, 2121, 278, 931, 259, 276, 283, 39660, 130972, 269, 259, 70232, 362, 259, 34418, 259, 122770, 870, 435, 13211, 286, 259, 26075, 2121, 1401, 337, 261, 319, 350, 5967, 440, 19441, 553, 25067, 426, 259, 34418, 259, 268, 128248, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,